Ставим Spacy для поиска сущностей

In [ ]:
!pip install spacy

Качаем исходники и словари с терминами

In [1]:
!git clone https://github.com/R-Vision/rvision-hackathon-2021-q1.git

Cloning into 'rvision-hackathon-2021-q1'...
remote: Enumerating objects: 1372, done.
remote: Counting objects: 100% (1372/1372), done.
remote: Compressing objects: 100% (1332/1332), done.
remote: Total 1372 (delta 40), reused 1371 (delta 39), pack-reused 0
Receiving objects: 100% (1372/1372), 15.43 MiB | 15.71 MiB/s, done.
Resolving deltas: 100% (40/40), done.


In [9]:
!git clone https://github.com/danil31219as/R-vision

Cloning into 'R-vision'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 15 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (15/15), done.


Качаем NLTK для предобработки

In [41]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

Прописываем пути

In [105]:
import os
import re
import spacy
import datetime
nlp = spacy.load("en_core_web_sm")
marked_path = '/content/rvision-hackathon-2021-q1/marked/'
dict_path = '/content/R-vision/dictionaries/'
convert_2020_path = '/content/rvision-hackathon-2021-q1/converted/2020/'

In [102]:
marked = os.listdir(marked_path)
convert = os.listdir(convert_2020_path)

In [14]:
tag_set = ['software', 'threat_actor', 'tool', 'attack_pattern', 'mitre_attack', 'country', 'ioc', 'malware', 'industry', 'created', 'date']

Чистка от стоп-слов

In [37]:
from nltk.corpus import stopwords
def clean_text(text):
  sw_eng = set(stopwords.words('english'))
  patterns = "[\n]+"
  text = re.sub(patterns,r' ',text)
  return ' '.join([word for word in text.split() if word.lower() not in sw_eng])

Лемматизация

In [39]:
from nltk import WordNetLemmatizer
from nltk import wordnet, pos_tag
def my_lemmatizer(sent):
    lemmatizer = WordNetLemmatizer()
    tokenized_sent = sent.split()
    pos_tagged = [(word, get_wordnet_pos(tag))
                 for word, tag in pos_tag(tokenized_sent)]
    return ' '.join([lemmatizer.lemmatize(word, tag)
                    for word, tag in pos_tagged])

In [43]:
def get_wordnet_pos(treebank_tag):
    my_switch = {
        'J': wordnet.wordnet.ADJ,
        'V': wordnet.wordnet.VERB,
        'N': wordnet.wordnet.NOUN,
        'R': wordnet.wordnet.ADV,
    }
    for key, item in my_switch.items():
        if treebank_tag.startswith(key):
            return item
    return wordnet.wordnet.NOUN

Ищем сущности времени и места

In [87]:
def set_country_time(text):
  countries = False
  timestamp = ''
  timestamp_2 = ''
  country = ''
  for ent in nlp(text).ents:
    
    if (ent.label_ == 'DATE' or ent.label_ == 'TIME') and len(ent.text) > len(timestamp) and re.findall('[0-9]+', ent.text):
      timestamp = ent.text
    if (ent.label_ == 'DATE' or ent.label_ == 'TIME') and len(ent.text) > len(timestamp_2) and ent.text != timestamp:
      timestamp_2 = ent.text
    elif ent.label_ == 'GPE' and not countries:
      country = ent.text
      countries = True
  if timestamp and len(timestamp) < 12:
    return [timestamp, country]
  return [timestamp_2, country]

Ищем IOC по паттерну

In [27]:
def set_ioc(text):
  iocs = []
  for ioc in re.findall('[0-9]+[a-z]+\w*', text):
    if ioc and len(ioc) > 10:
      iocs.append(ioc)
  return iocs

Malware по словарю терминов + шаблон

In [137]:
def set_malware(text):
  malware_list = open(dict_path + 'malware_list.txt').read().split('\n')
  malwares = []
  for word in text.split():
    if word.lower() in malware_list:
      malwares.append(word)
  for word in text.split():
    if re.findall('\w+[.]+\w+[.]+', word) and not word.startswith('http') and not word.startswith('www'):
      malwares.append(word)
  return malwares

Software из словаря терминов

In [29]:
def set_software(text):
  software_list = open(dict_path + 'software_list.txt').read().lower().split('\n')
  softwares = []
  for soft in software_list:
    if soft and soft in text.lower().split():
      softwares.append(soft)
    else:
      flag = True
      for elem in soft.split():
        if elem not in text.lower().split():
          flag = False
      if soft and flag:
        softwares.append(soft)
  return softwares

Код mitre_attack по паттерну

In [30]:
def set_mitre_attack(text, malwares):
  mitres = []
  for attack in re.findall('[T]+[0-9]+\w*', text):
    if attack and attack not in malwares:
      mitres.append(attack)
  return mitres

Threat_actor по словарю терминов

In [90]:
def set_threat_actor(text):
  threat_actors = open(dict_path + 'threat_actors.txt').read().lower().split('\n')
  actors = []
  for actor in threat_actors:
    if actor and actor in text.lower().split():
      actors.append(actor)
    else:
      flag = True
      for elem in actor.split():
        if elem not in text.lower().split():
          flag = False
      if flag and actor:
        actors.append(actor)
  return actors

Tool и Attack_pattern из словарей

In [32]:
def set_tool(text):
  tool_list = open(dict_path + 'tools.txt').read().lower().split('\n')
  tools = []
  for tool in tool_list:
    if tool and tool in text.lower().split():
      tools.append(tool)
    else:
      flag = True
      for elem in tool.split():
        if elem not in text.lower().split():
          flag = False
      if tool and flag:
        tools.append(tool)
  return tools

In [33]:
def set_attack_pattern(text):
  attack_pattern_list = open(dict_path + 'attack_patterns.txt').read().lower().split('\n')
  attack_patterns = []
  for attack_pattern in attack_patterns:
    if attack_pattern and attack_pattern in text.lower().split():
      attack_patterns.append(attack_pattern)
    else:
      flag = True
      for elem in attack_pattern.split():
        if elem not in text.lower().split():
          flag = False
      if attack_pattern and flag:
        attack_patterns.append(attack_pattern)
  return attack_patterns

Собираем объекты и сохраняем в json

In [108]:
objects = []
for i in range(len(convert)):
  d_tags = {}
  text = open(convert_2020_path + convert[i]).read()
  text = clean_text(text)
  text = my_lemmatizer(text)
  d_tags['ioc'] = list(set(set_ioc(text)))[:10]
  d_tags['date'], d_tags['country'] = set_country_time(text)
  d_tags['malware'] = list(set(set_malware(text)))[:10]
  d_tags['software'] = list(set(set_software(text)))[:10]
  d_tags['mitre_attack'] = list(set(set_mitre_attack(text, d_tags['malware'])))[:10]
  d_tags['threat_actor'] = list(set(set_threat_actor(text)))[:10]
  d_tags['tool'] = list(set(set_tool(text)))[:10]
  d_tags['attack_pattern'] = list(set(set_attack_pattern(text)))[:10]
  d_tags['created'] = str(datetime.datetime.today())
  objects.append(d_tags)

In [136]:
objects[10]

{'attack_pattern': [],
 'country': 'North Korea',
 'created': '2021-01-30 22:35:40.346043',
 'date': '13c47e19182454efa60890656244ee11c76b4904 2020',
 'ioc': ['7933716892e0d6053057f5f2df0ccadf5b06dc739fea79ee533dd0cec98ca971',
  '70f66e3131cfbda4d2b82ce9325fed79e1b3c7186bdbb5478f8cbd49b965a120',
  '23ec1c1d2182',
  '13c47e19182454efa60890656244ee11c76b4904',
  '659c854bbdefe692ee8c52761e7a8c7ee35aa56c',
  '35577959f79966b01f520e2f0283969155b8f8d7',
  '1e2a9367338d185ef477acc4d91ad45f5e6a7d11936c3eb4be463ae0b119185',
  '16d421807502a0b2429160e0bd960fa57f37efc4',
  '3847f5de342632f8f9e2901f16b7127472493ae',
  '6a3446b8a47f0ab4f536015218b22653fff8b18c595fbc5b0c09d857eba7c7a1'],
 'malware': ['saemaeul.mireene.com',
  'KONNI',
  'hxxps://web.opendrive.com/api/v1/download/file.json/MzBfMjA1Njc0ODhf?inline=0',
  '***_ECS_EPM.docx.',
  'orblog.mireene.com',
  'sgmedia.mireene.com',
  'hxxp://saemaeul.mireene.com/skin/visit/basic/log',
  'jmable.mireene.com',
  '10.8.11.9',
  'od.lk.'],
 'mitre